### Modelos
Modelo Óbito-Curado (60/40): Regressão e Modelo Internado-Curado (70/30): AdaBoost

In [44]:
import pandas as pd
import numpy as np

import import_ipynb
import modelofuncoes as mf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [31]:
# Carregando a Base de Dados

url = 'Vacinas/OC/oc-vacina-6040-sp.xlsx'
alvo = 'evolucaoCaso'
baseObitoCurado = pd.read_excel(url)

bases = {}
bases['Base OC (60/40)'] = url

In [36]:
# Definindo o Modelo de Regressão Logística
hiperparametrosLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, 
                                       intercept_scaling=1, l1_ratio=None, max_iter=1000, multi_class='auto', 
                                       n_jobs=None, penalty='l2', random_state=1340, solver='lbfgs', 
                                       tol=0.0001, verbose=0, warm_start=False)

modeloClassificador = {"LR": hiperparametrosLR}

In [37]:
valoresAlvo = [0, 1]

colunasRemovidas = ['dataPrimeiraDose', 'dataSegundaDose', 'assintomatico', 'puerpera', 'disturbiosGustatorios', 
                    'disturbiosOlfativos', 'gestante', 'tosse', 'respiratoria', 'fragilidadeImuno', 'racaCor', 
                    'sexo', 'febre', 'renal', 'imunossupressao', 'obesidade']

desempenhoDosModelos = mf.testarModelos(bases, alvo, colunasRemovidas, modeloClassificador, valoresAlvo)
desempenhoDosModelos

Index(['evolucaoCaso', 'dorDeGarganta', 'dispneia', 'coriza', 'dorDeCabeca',
       'diabetes', 'cardiaca', 'faixaetaria', 'qntVacinas'],
      dtype='object')


,Base,Modelo,Hiperparâmetros,Acurácia,Precisão (0),Recall (0),F1-Score (0),Precisão (1),Recall (1),F1-Score (1),3 Kfolds,5 Kfolds
0,Base OC (60/40),LR,"{'C': 1.0, 'class_weight': None, 'dual': False...",83.372%,84.181%,56.546%,67.65%,83.162%,95.282%,88.81%,83.473%,83.806%


### Desempenho - Treinamento e Teste

In [40]:
# Carregando a Base de Dados
baseObitoCurado = pd.read_excel(url)

In [41]:
# Selecionando os Atributos do Pré-Processamento

atributosSelecionados = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
                         'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca', 'evolucaoCaso']

baseObitoCurado = baseObitoCurado.loc[:, atributosSelecionados]

In [42]:
x_train, x_test, y_train, y_test = train_test_split(baseObitoCurado.drop(alvo,axis=1),
                                                    baseObitoCurado[alvo],
                                                    test_size=0.3,
                                                    random_state=42);

In [45]:
# Fit nos dados
modeloClassificador = hiperparametrosLR
modeloClassificador.fit(x_train, y_train)

#Realizando a predição
resultadoTeste = modeloClassificador.predict(x_test)

# Verificando a acurácia
print(metrics.classification_report(y_test,resultadoTeste))

              precision    recall  f1-score   support

           0       0.84      0.57      0.68       527
           1       0.83      0.95      0.89      1187

    accuracy                           0.83      1714
   macro avg       0.84      0.76      0.78      1714
weighted avg       0.83      0.83      0.82      1714



### Avaliando as outras Bases 

In [46]:
url = 'Vacinas/OC/oc-vacina-6040-rj.xlsx'
baseTeste = pd.read_excel(url)

In [47]:
colunasSelecionadasTeste = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
              'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca']

dadosTeste = baseTeste.loc[:, colunasSelecionadasTeste]
dadosAlvo =  baseTeste.loc[:, alvo]

In [53]:
# Coletando as variáveis e atributo alvo
predicaoResultado = modeloClassificador.predict(dadosTeste)
print(metrics.classification_report(dadosAlvo, predicaoResultado))

              precision    recall  f1-score   support

           0       0.80      0.65      0.72       585
           1       0.87      0.94      0.90      1462

    accuracy                           0.85      2047
   macro avg       0.84      0.79      0.81      2047
weighted avg       0.85      0.85      0.85      2047

